# SSEBop Single Image Example

In [1]:
import ee
from IPython.display import Image

from ssebop import ssebop
from ssebop import landsat
ssebop = reload(ssebop)
landsat = reload(landsat)

ee.Initialize()

ndvi_palette = ['#EFE7E1', '#003300']
# ta_palette = ['#ff0000', '#ffff00', '#00ffff', '#0000ff']
et_palette = [
  'DEC29B', 'E6CDA1', 'EDD9A6', 'F5E4A9', 'FFF4AD', 'C3E683', '6BCC5C', 
  '3BB369', '20998F', '1C8691', '16678A', '114982', '0B2C7A']

#### Prep the image

In [2]:
# scene_id = 'LC08_038038_20150717'
landsat_img = ee.Image('LANDSAT/LC08/C01/T1_RT_TOA/LC08_043033_20150602')
landsat_img = landsat_img \
    .select(
        ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10'],
        ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'thermal']) \
    .set('k1_constant', landsat_img.get('K1_CONSTANT_BAND_10')) \
    .set('k2_constant', landsat_img.get('K2_CONSTANT_BAND_10'))
input_img = ee.Image(landsat.prep_func(landsat_img))

#### Study area and region
Needed for making Ta thumbnail match Landsat thumbnails

In [3]:
image_region = landsat_img.geometry().bounds(1, 'EPSG:4326').coordinates().getInfo()
image_crs = landsat_img.select([1]).projection().crs().getInfo()
image_transform = landsat_img.select([1]).projection().getInfo()['transform']
coarse_transform = image_transform[:]
coarse_transform[0] = 240
coarse_transform[4] = 240

#### Show NDVI

In [4]:
Image(url=input_img.select(['ndvi']).getThumbURL({
    'min': -0.1, 'max': 0.9, 'palette': ','.join(ndvi_palette)}))

#### Setup DisALEXI

In [5]:
ssebop_obj = ssebop.SSEBop(
    input_img, 
    k_factor=1.25,
    dt_source='ASSET',
    elev_source='ASSET',
    tcorr_source='SCENE',
    tmax_source='DAYMET',
    tdiff_buffer_value=10)

#### Compute ET

In [6]:
# Projecting to the Landsat image for display purposes
et_img = ssebop_obj.compute_etf() \
    .reproject(crs=image_crs, crsTransform=coarse_transform)
Image(url=et_img.getThumbURL({
    'min': 0, 'max': 1.25, 'region': image_region, 'palette': ','.join(et_palette)}))